In [503]:
%gui qt6
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution, dual_annealing, minimize, least_squares
import os
from sklearn.metrics import root_mean_squared_error
import pyqtgraph as pg
import PyQt6 as p6
from numba import jit
np.seterr(all='ignore')

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [310]:
def eV_to_um(eV):
    return 1.2398/eV

def um_to_eV(um):
    return 1.2398/um

In [311]:
SiO2_nk = pd.read_csv(os.path.join('..', 'resources', 'SiO2_nk.csv'), sep='\t', header=1, index_col=3).reset_index(drop=True)


In [312]:
SiO2_nk

,um,n,k
0,0.250,1.513525,0.0
1,0.255,1.510877,0.0
2,0.260,1.508413,0.0
3,0.265,1.506117,0.0
4,0.270,1.503972,0.0
...,...,...,...
146,0.980,1.455662,0.0
147,0.985,1.455593,0.0
148,0.990,1.455524,0.0
149,0.995,1.455455,0.0


In [313]:
def cauchy(parameters, wavelength):
    A, B, C = parameters
    _array = A + B/wavelength**2 + C/wavelength**4
    return _array

In [314]:
wavelength = np.array(SiO2_nk.iloc[:,0]).reshape(SiO2_nk.iloc[:,0].shape[0])
ph_energy = np.array(um_to_eV(SiO2_nk.iloc[:,0])).reshape(SiO2_nk.iloc[:,0].shape[0])
y_true = np.array(SiO2_nk.iloc[:,1]).reshape(SiO2_nk.iloc[:,1].shape[0])

In [315]:
def cauchy_plot(parameters, wavelength):
    plt = pg.plot()
    plt.showGrid(x=True, y=True)
    plt.addLegend()
    _cauchy = cauchy(parameters, wavelength)#.reshape(-1)
    plt.plot(wavelength, _cauchy, pen=pg.mkPen('red', width=1.5), name='n')
    plt.setLabel('bottom', 'Wavelength', units='μm')
    plt.setWindowTitle('Refractive index plot')
    plt.setBackground('w')


def nk_plot(wavelength, n, k = None):
    plt = pg.plot()
    plt.showGrid(x=True, y=True)
    plt.addLegend()
    #eV = np.array(eV).reshape(151)
    n = np.array(n).reshape(151)
    plt.setLabel('bottom', 'Wavelength', units='μm')
    plt.plot(wavelength, n, pen=pg.mkPen('red', width=1.5), name='n')
    if k is not None:
        plt.plot(wavelength, k, pen=pg.mkPen('blue', width=1.5), name='k')

    plt.setWindowTitle('Refractive index plot')
    plt.setBackground('w')
    

def e_plot(ph_energy, e1, e2):
    plt = pg.plot()
    plt.showGrid(x=True, y=True)
    plt.addLegend()
    plt.setLabel('bottom', 'Photon energy', units='eV')
    plt.plot(ph_energy, e1, pen=pg.mkPen('red', width=1.5), name='e1')
    plt.plot(ph_energy, e2, pen=pg.mkPen('green', width=1.5), name='e2')
    '''if wavelength:
    else:
        plt.setLabel('bottom', 'Photon energy', units='eV')
        plt.plot(eV, n, pen=pg.mkPen('red', width=1.5), name='n')
        if k is not None:
            plt.plot(eV, k, pen=pg.mkPen('blue', width=1.5), name='k')

    '''
    plt.setWindowTitle('e plot')
    plt.setBackground('w')

def compare_results(x_space, y_pred, y_true):
    plt = pg.plot()
    plt.showGrid(x=True, y=True)
    plt.addLegend()
    #plt.setLabel('bottom', 'Wavelength')
    if len(y_pred.shape) > 1:
        for i in range(y_pred.shape[1]):
            plt.plot(x_space, y_pred[:, i], pen=pg.mkPen(color=(i), width=1.5), name=f'prediction {i}')
    else:
        plt.plot(x_space, y_pred, pen=pg.mkPen('red', width=1.5), name=f'prediction')

    if len(list(y_true.shape)) > 1:
        for j in range(y_true.shape[1]):
            plt.plot(x_space, y_true[:, j], pen=pg.mkPen((i+1+j), width=1.5), name=f'true {j}')
    else:
        plt.plot(x_space, y_true, pen=pg.mkPen('blue', width=1.5), name=f'true')
    

    plt.setWindowTitle('Refractive index plot')
    plt.setBackground('w')

In [316]:
def TL_e1(parameters, ph_energy):
    A, E0, C, Eg, einf = parameters
    alpha_cp = 4 * (E0**2) - (C**2)
    gamma_cp = (E0**2) - ((C ** 2) / 2)
    alpha = np.sqrt(alpha_cp)
    gamma = np.sqrt(gamma_cp)
    a_ln = ((Eg**2) - (E0**2)) * (ph_energy**2) + (Eg**2 * C**2) - (E0**2 * (E0**2 + 3*Eg**2))
    a_tan = ((ph_energy**2) - (E0**2)) * ((E0**2) + (Eg**2)) + ((Eg**2) * (C**2))
    dzeta4 = ((ph_energy**2) - (gamma**2))**2 + (((alpha**2) * (C**2))/4)

    e1 = einf + \
    (((A*C)/(np.pi*dzeta4)) * (a_ln/(2*alpha*E0)) * np.log(((E0**2) + (Eg**2) + (alpha*Eg))/((E0**2) + (Eg**2) - (alpha*Eg)))) \
    - ((A/(np.pi*dzeta4)) * (a_tan/E0)*(np.pi - np.arctan((2*Eg+alpha)/C) + np.arctan(((-2*Eg) + alpha)/C))) \
    + (2*((A*E0)/(np.pi*dzeta4*alpha)) * Eg*((ph_energy**2) - (gamma**2))*(np.pi + 2*np.arctan(2*((gamma**2) - (Eg**2))/(alpha*C)))) \
    - (((A*E0*C)/(np.pi*dzeta4)) * (((ph_energy**2) + (Eg**2))/ph_energy) * np.log(abs(ph_energy-Eg)/(ph_energy+Eg)))\
    + ((2*A*E0*C)/(np.pi*dzeta4)) *Eg * np.log((abs(ph_energy-Eg)*(ph_energy+Eg))/(np.sqrt(((E0**2) - (Eg**2))**2 + ((Eg**2) * (C**2)))))
    e1 = np.nan_to_num(e1)
    return np.where((alpha_cp <= 0 or gamma_cp <= 0 or E0 <= 0 or C <= 0 or ((E0**2 + Eg**2 - alpha*Eg ) < 0)), 0, e1)

def TL_e2(parameters, ph_energy):
    A, E0, C, Eg = parameters
    e2 = ((A*E0*C*((ph_energy-Eg)**2)) / (((ph_energy**2) - (E0**2))**2 + ((C**2)*ph_energy**2))) * (1/ph_energy)
    return np.where(ph_energy > Eg, e2, 0)

def TL(parameters, ph_energy):
    '''
        A

        E0

        C

        Eg

        einf
    '''
    return np.stack([TL_e1(parameters, ph_energy), TL_e2(parameters[:-1], ph_energy)], axis=1)

In [317]:
def multi_model(parameters, *args):
    '''
    args:
        x_space
        
        functions
        
        param_sizes
    '''
    x_space = args[0] #ph_energy or wavelength
    functions = args[1]
    param_sizes = args[2]

    y_pred = np.sum([functions[i](parameters[sum(param_sizes[:i]): sum(param_sizes[:i+1])], x_space) for i in range(len(functions))], axis=0)

    return y_pred
    

In [318]:
def fitness_func(parameters, *args):
    '''
    args:
        x_space

        y_true
        
        functions
        
        param_sizes
    '''
    x_space = args[0] #ph_energy or wavelength
    y_true = args[1]
    functions = args[2]
    param_sizes = args[3]
    if len(functions) > 1:
        y_pred = multi_model(parameters, x_space, functions, param_sizes)
        #np.sum([functions[i](parameters[sum(param_sizes[:i]): sum(param_sizes[:i+1])], x_space) for i in range(len(functions))], axis=0)
    else:
        y_pred = functions[0](parameters, x_space)
        
    #loss = np.where(np.isnan(y_pred).any(), 5, root_mean_squared_error(y_true, y_pred))
    if np.isnan(y_pred).any():
        return 100
    else:
        loss = root_mean_squared_error(y_true, y_pred)
        return loss

In [319]:
Si_e = pd.read_csv(os.path.join('..', 'resources', 'Si_e.csv'), sep='\t', header=1,index_col=3).reset_index(drop=True)
Si_e

,eV,e1,e2
0,4.960000,-10.647780,12.186640
1,4.862745,-11.955365,13.009671
2,4.769231,-13.629112,14.336851
3,4.679246,-15.588576,16.423994
4,4.592593,-17.702778,19.582952
...,...,...,...
146,1.265306,12.841301,0.003723
147,1.258883,12.827988,0.003429
148,1.252525,12.814975,0.003155
149,1.246231,12.802176,0.002896


In [320]:
TL_e2_ = Si_e.loc[:,['e2']].to_numpy()
TL_e2_.shape

(151, 1)

In [15]:
osc_number = 3
multi_TL_finder2 = differential_evolution(
    fitness_func,
    #[bound for _ in range(1) for bound in [(0.001,1000), (3,4), (0.001,5), (1,2), (0.001,2), (0.001, 1000), (3.5,4.5), (0.001,5), (2,4), (0.001,2), (0.001,1000), (4,5), (0.001,5), (2,4), (0.001,2)]],
    [bound for _ in range(osc_number) for bound in [(0.001, 1000), (3,5), (0.001,5), (2,4), (0.5,2)]],
    args=(test_ph_energy, TL_true, [TL for _ in range(osc_number)], [5 for _ in range(osc_number)]),
    init='sobol',
    #maxiter=200,
    polish=True, 
    #updating='deferred',
    #atol=1
    #x0=[996, 3.36, 0.408, 3.02, 1, 892, 8.73, 2.95, 6.49, 1, 3.89, 4.20, 0.61, 1.06, 1]
)

In [16]:
print(multi_TL_finder2.x, multi_TL_finder2.fun)

[1.57484352e+02 3.82172771e+00 1.01881489e+00 2.39645246e+00
 5.04548859e-01 4.04984578e+01 4.24217370e+00 3.71531836e-01
 2.15629477e+00 5.41621302e-01 8.29828757e+02 3.36319620e+00
 2.50098688e-01 3.08230932e+00 5.12107476e-01] 0.24333511858801993


In [17]:
compare_results(test_ph_energy, y_pred=multi_model(multi_TL_finder2.x, test_ph_energy, [TL for _ in range(osc_number)], [5 for _ in range(osc_number)]), y_true=TL_true)

In [321]:
TiO2 = pd.read_csv(os.path.join('..', 'resources', 'TiO2.csv'), sep='\t', index_col=3).reset_index(drop=True)

In [322]:
TiO2

,eV,e1,e2
0,4.95,3.959499,5.979951
1,4.90,4.168708,6.052847
2,4.85,4.392638,6.116233
3,4.80,4.631594,6.168044
4,4.75,4.885654,6.205946
...,...,...,...
70,1.45,4.810812,0.000000
71,1.40,4.795345,0.000000
72,1.35,4.780606,0.000000
73,1.30,4.766572,0.000000


In [202]:
osc_number = 1
multi_TL_finder = differential_evolution(
    fitness_func,
    [bound for _ in range(osc_number) for bound in [(0.001,1000), (4,7), (0.001,5), (3.2,3.5), (0,10)]],
    #[bound for _ in range(osc_number) for bound in [(50, 500), (3.2,4.3), (0.2,1), (2.3,3.1), (1.4,1.6)]],
    args=(TiO2.loc[:,'eV'].to_numpy(), TiO2.loc[:,['e1', 'e2']].to_numpy(), [TL for _ in range(osc_number)], [5 for _ in range(osc_number)]),
    #init='random',
    init='sobol',
    #popsize=100,
    #maxiter=200,
    polish=True,
    updating='deferred',
    #strategy='best2exp',
    #x0=[250, 4, 1.7, 3.3,1]
    #mutation=1.9, 
    #recombination=1
    #workers=2
)

In [204]:
compare_results(TiO2.loc[:,'eV'], multi_model(multi_TL_finder.x, TiO2.loc[:,'eV'], [TL for _ in range(osc_number)], [5 for _ in range(osc_number)]), TiO2.loc[:,['e1', 'e2']].to_numpy())

In [235]:
osc_number = 1
SiO2_finder = differential_evolution(
    fitness_func,
    [bound for _ in range(osc_number) for bound in [(1.3,1.55), (1e-5,0.1), (1e-7,0.1)]],
    args=(SiO2_nk.loc[:,'um'].to_numpy(), SiO2_nk.loc[:,'n'].to_numpy(), [cauchy for _ in range(osc_number)], [3 for _ in range(osc_number)]),
    init='sobol',
    polish=True,
    updating='deferred'
)

In [236]:
print(SiO2_finder.x, SiO2_finder.fun)

[1.45263457e+00 3.70238615e-03 2.30783493e-06] 0.00040269172631006325


In [225]:
nk_plot(SiO2_nk.loc[:,'um'].to_numpy(), cauchy(SiO2_finder.x, SiO2_nk.loc[:,'um'].to_numpy()))

In [237]:
compare_results(SiO2_nk.loc[:,'um'].to_numpy(), multi_model(SiO2_finder.x, SiO2_nk.loc[:,'um'].to_numpy(), [cauchy for _ in range(osc_number)], [3 for _ in range(osc_number)]), SiO2_nk.loc[:,'n'].to_numpy())

In [323]:
H2O_nk = pd.read_csv(os.path.join('..', 'resources', 'H2O.csv'), sep='\t', index_col=3).reset_index(drop=True)

In [324]:
H2O_nk

,um,n,k
0,0.250505,1.396272,0.0
1,0.253061,1.394556,0.0
2,0.255670,1.392870,0.0
3,0.258333,1.391213,0.0
4,0.261053,1.389585,0.0
...,...,...,...
69,0.826667,1.325848,0.0
70,0.855172,1.325153,0.0
71,0.885714,1.324436,0.0
72,0.918518,1.323693,0.0


In [325]:
def fit(x, y, functions, functions_parameters_number, bounds):
    finder = differential_evolution(
        func=fitness_func,
        bounds= bounds,
        args=(x, y, functions, functions_parameters_number),
        init='sobol',
        polish=True
    )
    return finder.x, finder.fun

In [326]:
def Sellmeier(parameters, wavelength):
    n = np.sqrt(1 + (parameters[0]*wavelength**2)/(wavelength**2 - parameters[1]) 
                + (parameters[2]*wavelength**2)/(wavelength**2 - parameters[3]))
    return n

In [72]:
fit(
    H2O_nk['um'],
    H2O_nk['n'],
    [Sellmeier],
    [4],
    [(0.001,10), (0.001,10), (0.001,10), (0.001,10)]
)

(array([0.18233643, 9.99439735, 0.75667223, 0.0128541 ]),
 0.00012284861778764995)

In [67]:
osc_number = 1
H2O_finder = differential_evolution(
    fitness_func,
    [bound for _ in range(osc_number) for bound in [(0.001,10), (0.001,10), (0.001,10), (0.001,10)]],
    args=(H2O_nk.loc[:,'um'], H2O_nk.loc[:,'n'], [Sellmeier for _ in range(osc_number)], [4 for _ in range(osc_number)]),
    init='sobol',
    polish=True,
)

In [68]:
print(H2O_finder.x, H2O_finder.fun)

[0.18179712 9.9875776  0.7566564  0.01285528] 0.00012284554313388214


In [69]:
compare_results(H2O_nk.loc[:,'um'].to_numpy(), multi_model(H2O_finder.x, H2O_nk.loc[:,'um'].to_numpy(), [Sellmeier for _ in range(osc_number)], [4 for _ in range(osc_number)]), H2O_nk.loc[:,'n'].to_numpy())

In [486]:
def theta_j(N_i, N_j, theta_i):
    return np.arcsin((N_i*np.sin(theta_i))/N_j)

def beta(thickness, wavelength, N, theta):
    return (2*np.pi*thickness)/wavelength * N * np.cos(theta)

def N(n,k):
    return n-1j*k

def e_to_n(e1, e2):
    return np.sqrt((e1 + np.sqrt(e1**2 + e2**2))/2)

def e_to_k(e1, e2):
    return np.sqrt((-e1 + np.sqrt(e1**2 + e2**2))/2)

In [538]:
def rs(N_i=None, N_j=None, theta_i=None, theta_j=None, Ss=None):
    if Ss is not None:
        #print(Ss)
        return Ss[1,0]/\
               Ss[0,0]
    else:
        return ((N_i * np.cos(theta_i))-(N_j * np.cos(theta_j)))/\
               ((N_i * np.cos(theta_i))+(N_j * np.cos(theta_j)))

def rp(N_i=None, N_j=None, theta_i=None, theta_j=None, Sp=None):
    if Sp is not None:
        return Sp[1,0]/Sp[0,0]
    else:
        return ((N_j * np.cos(theta_i))-(N_i * np.cos(theta_j)))/\
               ((N_j * np.cos(theta_i))+(N_i * np.cos(theta_j)))
    
def t_s(N_i, N_j, theta_i, theta_j):
    return (2*N_i*np.cos(theta_i))/\
        (N_i*np.cos(theta_i)+N_j*np.cos(theta_j))

def t_p(N_i, N_j, theta_i, theta_j):
    return (2*N_i*np.cos(theta_i))/\
        (N_j*np.cos(theta_i)+N_i*np.cos(theta_j))

In [522]:
def I_matrix(r, t):
    return np.array(
        [[1, r], [r, 1]]) / t

def L_matrix(beta):
    return np.array(
            [[np.exp(-1j*beta), 0],
                    [0, np.exp(1j*beta)]]
    )

def S_matrix(I_matrixes, L_matrixes):
    S = np.identity(2)
    S = np.matmul(S, I_matrixes[0])
    #S = I_matrixes[0]
    for i in range(1, len(L_matrixes)):
        S = np.matmul(S, L_matrixes[i])
        S = np.matmul(S, I_matrixes[i])
    return S

def M_n(beta, r, t):
    return np.array([[np.exp(-1j*beta), 0], [0, np.exp(1j*beta)]]) @ np.array([[1, r], [r, 1]]) / t

def M(r01, t01, M_ns):
    M = (np.array([[1, r01], [r01, 1]]) / t01)
    for M_n in M_ns:
        M = M @ M_n
    return M

In [721]:
def psi(r_p, r_s):
    return np.arctan(np.abs(r_p/r_s))

def delta(r_p, r_s):
    angle = np.angle(r_p/r_s)
    return np.where(angle < 0, angle + 2*np.pi, angle)
    #return angle

In [331]:
def psi_delta_plot(x_space, psi, delta):
    plt = pg.plot()
    plt.showGrid(x=True, y=True)
    plt.addLegend()
    plt.plot(x_space, psi, pen=pg.mkPen('red', width=1.5), name='Psi')
    plt.plot(x_space, delta, pen=pg.mkPen('green', width=1.5), name='Delta')
    plt.setWindowTitle('Psi delta plot')
    plt.setBackground('w')

In [385]:
Si_psi_delta = pd.read_csv(os.path.join('..', 'resources', 'Si_psi_delta.csv'), sep='\t', index_col=3).reset_index(drop=True)

In [386]:
Si_psi_delta

,Wavelength,Psi,Delta
0,0.250,37.499027,142.067261
1,0.255,37.954182,143.491226
2,0.260,38.424641,145.358765
3,0.265,38.753490,147.528564
4,0.270,38.906830,150.029648
...,...,...,...
146,0.980,21.694263,180.036819
147,0.985,21.646151,179.941025
148,0.990,21.700411,180.030838
149,0.995,21.628141,179.945282


In [387]:
psi_delta_plot(Si_psi_delta['Wavelength'], Si_psi_delta['Psi'], Si_psi_delta['Delta'])

In [388]:
Si_e

,eV,e1,e2
0,4.960000,-10.647780,12.186640
1,4.862745,-11.955365,13.009671
2,4.769231,-13.629112,14.336851
3,4.679246,-15.588576,16.423994
4,4.592593,-17.702778,19.582952
...,...,...,...
146,1.265306,12.841301,0.003723
147,1.258883,12.827988,0.003429
148,1.252525,12.814975,0.003155
149,1.246231,12.802176,0.002896


In [389]:
Si_e_n = e_to_n(Si_e['e1'], Si_e['e2'])
Si_e_k = e_to_k(Si_e['e1'], Si_e['e2'])
Si_N = N(Si_e_n, Si_e_k)
air_N = np.array([1 for _ in range(len(Si_N))])

In [390]:
theta_i = np.deg2rad(60)
Si_rp = rp(air_N, Si_N, theta_i, theta_j(air_N, Si_N, theta_i))
Si_rs = rs(air_N, Si_N, theta_i, theta_j(air_N, Si_N, theta_i))

In [405]:
Si_psi = np.rad2deg(psi(Si_rp, Si_rs))
Si_delta = np.rad2deg(delta(Si_rp, Si_rs))

In [406]:
root_mean_squared_error([Si_psi_delta['Psi'], Si_psi_delta['Delta']], [np.rad2deg(psi(Si_rp, Si_rs)), np.rad2deg(delta(Si_rp, Si_rs))])

0.04741109762859936

In [407]:
psi_delta_plot(Si_psi_delta['Wavelength'], Si_psi, Si_delta)

In [408]:
Si_50nmSiO2_psi_delta = pd.read_csv(os.path.join('..', 'resources', 'Si_50nmSiO2_psi_delta.csv'), sep='\t', index_col=3).reset_index(drop=True)

In [395]:
Si_50nmSiO2_psi_delta

,Wavelength,Psi,Delta
0,0.250,47.638897,207.949005
1,0.255,48.485065,202.943573
2,0.260,49.256603,197.032501
3,0.265,49.837082,190.267899
4,0.270,50.195648,182.787872
...,...,...,...
146,0.980,23.210943,147.138138
147,0.985,23.237051,147.315887
148,0.990,23.244204,147.566055
149,0.995,23.184164,147.629013


In [665]:
psi_delta_plot(Si_50nmSiO2_psi_delta['Wavelength'], Si_50nmSiO2_psi_delta['Psi'], Si_50nmSiO2_psi_delta['Delta'])

In [397]:
Si_nk = pd.read_csv(os.path.join('..', 'resources', 'Si_nk.csv'), sep='\t', index_col=3).reset_index(drop=True)

In [398]:
Air_nk = pd.DataFrame([[1,0] for _ in range(len(Si_nk))], columns=['n', 'k'])

In [399]:
Ns = np.array([N(Air_nk['n'], Air_nk['k']), N(SiO2_nk['n'], SiO2_nk['k']), N(Si_nk['n'], Si_nk['k'])])

In [487]:
Ns.shape

(3, 151)

In [739]:
thicknesses = [0,0.05, 100]
#thetas = [np.deg2rad(60)]
psis = []
deltas = []
for idx, wavelength in enumerate(Si_50nmSiO2_psi_delta['Wavelength']):
    thetas = [np.deg2rad(60)]
    for idx_layer in range(len(Ns)-1):
        thetas.append(theta_j(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[-1]))
    #print(thetas)

    Is_matrixes = np.array([I_matrix(rs(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[idx_layer], theta_j(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[idx_layer])), t_s(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[idx_layer], theta_j(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[idx_layer]))) for idx_layer in range(len(Ns)-1)])

    Ip_matrixes = np.array([I_matrix(rp(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[idx_layer], theta_j(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[idx_layer])), t_p(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[idx_layer], theta_j(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[idx_layer]))) for idx_layer in range(len(Ns)-1)])

    L_matrixes = np.array([L_matrix(beta(thicknesses[idx_layer-1], wavelength, Ns[idx_layer][idx], thetas[idx_layer])) for idx_layer in range(len(Ns)-1)])
    #print(Is_matrixes*L_matrixes)



    Ss = S_matrix(Is_matrixes, L_matrixes)
    Sp = S_matrix(Ip_matrixes, L_matrixes)

    #print(Sp)

    rp_ = rp(Sp=Sp)
    rs_ = rs(Ss=Ss)
    #print(rs_)

    psis.append(psi(rp_, rs_))
    deltas.append(delta(rp_, rs_))

#L_matrixes.shape
Is_matrixes.shape


(2, 2, 2)

In [732]:
thicknesses = [1,0.05, 1]
#thetas = [np.deg2rad(60)]
psis = []
deltas = []
for idx, wavelength in enumerate(Si_50nmSiO2_psi_delta['Wavelength']):
    thetas = [np.deg2rad(60)]
    for idx_layer in range(len(Ns)-1):
        thetas.append(theta_j(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[-1]))
    #print(thetas)

    _M_np = [M_n(beta(thicknesses[i], wavelength, Ns[i][idx], thetas[i]), rp(Ns[i][idx], Ns[i+1][idx], thetas[i], thetas[i+1]), t_p(Ns[i][idx], Ns[i+1][idx], thetas[i], thetas[i+1])) for i in range(1, len(Ns)-2)]

    _M_ns = [M_n(beta(thicknesses[i], wavelength, Ns[i][idx], thetas[i]), rs(Ns[i][idx], Ns[i+1][idx], thetas[i], thetas[i+1]), t_s(Ns[i][idx], Ns[i+1][idx], thetas[i], thetas[i+1])) for i in range(1, len(Ns)-2)]

    _Mp = M(rp(Ns[0][idx], Ns[1][idx], thetas[0], thetas[1]), t_p(Ns[0][idx], Ns[1][idx], thetas[0], thetas[1]), _M_np)

    _Ms = M(rs(Ns[0][idx], Ns[1][idx], thetas[0], thetas[1]), t_s(Ns[0][idx], Ns[1][idx], thetas[0], thetas[1]), _M_ns)

    #print(_Ms)

    #print(Sp)

    rp_ = rp(Sp=_Mp)
    rs_ = rs(Ss=_Ms)
    #print(rs_)

    psis.append(psi(rp_, rs_))
    deltas.append(delta(rp_, rs_))


In [740]:
deltas
psis

[0.6551209534769377,
 0.6626715381965567,
 0.6698975472247743,
 0.6754691424610086,
 0.678820547189935,
 0.6790309791142861,
 0.6731772749781129,
 0.6664059592693777,
 0.6528060670921145,
 0.6339116409909796,
 0.6174343529615725,
 0.6054265311833115,
 0.5969119701973539,
 0.5907426824385028,
 0.5861238584954996,
 0.5826060614591175,
 0.5799749134174037,
 0.5781973497522844,
 0.5773715364599729,
 0.5776825124522101,
 0.57946448557095,
 0.5834734077286586,
 0.5899129015875955,
 0.5941455877963487,
 0.5901579065750108,
 0.5792778663941507,
 0.5662125988230905,
 0.5534293451524035,
 0.5417585955887083,
 0.5315506284791928,
 0.5228392496581096,
 0.5154047049214485,
 0.5088802840068416,
 0.5029829270204225,
 0.4975551218813516,
 0.4924989134037953,
 0.48776010291106764,
 0.483295038480002,
 0.4790866585669038,
 0.4751083593837014,
 0.47134267142860303,
 0.4677763967065012,
 0.4643978441724534,
 0.4611953790928571,
 0.4581580896681794,
 0.45527697327685934,
 0.4525465554598747,
 0.44995164422

In [741]:
psi_delta_plot(Si_50nmSiO2_psi_delta['Wavelength'], np.rad2deg(psis), np.rad2deg(deltas))

In [776]:
def psi_delta_fitness(parameters, *args):
        '''
        #args:
                x_space

                functions

                param_sizes

                Ns

                thicknesses

                y_true
        '''
        x_space = args[0]
        functions = args[1]
        param_sizes = args[2]
        Ns = args[3]
        thicknesses = args[4]
        y_true = args[5]
        OC = multi_model(parameters[:-1], x_space, functions, param_sizes)
        #print(OC)
        Ns.insert(1, N(OC, np.array([0 for _ in range(len(OC))])))
        #print(Ns[2])
        thicknesses.insert(1, parameters[-1])
        psis = []
        deltas = []
        for idx, x in enumerate(x_space):
                thetas = [np.deg2rad(60)]
                for idx_layer in range(len(Ns)-1):
                        thetas.append(theta_j(Ns[idx_layer][idx], Ns[idx_layer+1][idx], thetas[-1]))

                r012p = (rp(Ns[0][idx], Ns[1][idx], thetas[0], thetas[1]) + rp(Ns[1][idx], Ns[2][idx], thetas[1], thetas[2])*np.exp(-2j*beta(thicknesses[1], x, Ns[1][idx], thetas[1])))/\
                        (1 + rp(Ns[0][idx], Ns[1][idx], thetas[0], thetas[1]) * rp(Ns[1][idx], Ns[2][idx], thetas[1], thetas[2])*np.exp(-2j*beta(thicknesses[1], x, Ns[1][idx], thetas[1])))

                r012s = (rs(Ns[0][idx], Ns[1][idx], thetas[0], thetas[1]) + rs(Ns[1][idx], Ns[2][idx], thetas[1], thetas[2])*np.exp(-2j*beta(thicknesses[1], x, Ns[1][idx], thetas[1])))/\
                        (1 + rs(Ns[0][idx], Ns[1][idx], thetas[0], thetas[1]) * rs(Ns[1][idx], Ns[2][idx], thetas[1], thetas[2])*np.exp(-2j*beta(thicknesses[1], x, Ns[1][idx], thetas[1])))

                psis.append(np.rad2deg(psi(r012p, r012s)))
                deltas.append(np.rad2deg(delta(r012p, r012s)))

        loss = root_mean_squared_error([psis, deltas], y_true)

        return loss

In [777]:
def fit_psi_delta(x_space, functions, param_sizes, Ns, thicknesses, y_true, bounds):
    finder = differential_evolution(
        func=psi_delta_fitness,
        bounds= bounds,
        args=(x_space, functions, param_sizes, Ns, thicknesses, y_true),
        init='sobol',
        polish=True
    )
    return finder.x, finder.fun

In [781]:
fitted1 = fit_psi_delta(
    Si_50nmSiO2_psi_delta['Wavelength'],
    [cauchy],
    [3],
    [N(Air_nk['n'], Air_nk['k']), N(Si_nk['n'], Si_nk['k'])],
    [1,1],
    [Si_50nmSiO2_psi_delta['Psi'], Si_50nmSiO2_psi_delta['Delta']],
    [(0.5,2), (1e-5, 1e-1), (1e-5, 1e-1), (0, 0.2)]
)

KeyboardInterrupt: 

In [615]:
Si_50nmSiO2_50nmTiO2_psi_delta = pd.read_csv(os.path.join('..', 'resources', 'Si_50nmSiO2_50nmTiO2_psi_delta.csv'), sep='\t', index_col=3).reset_index(drop=True)

In [616]:
Si_50nmSiO2_50nmTiO2_psi_delta['Delta'] = np.where(Si_50nmSiO2_50nmTiO2_psi_delta['Delta'] < 0, Si_50nmSiO2_50nmTiO2_psi_delta['Delta'] + 360, Si_50nmSiO2_50nmTiO2_psi_delta['Delta'])

In [627]:
psi_delta_plot(Si_50nmSiO2_50nmTiO2_psi_delta['Wavelength'], Si_50nmSiO2_50nmTiO2_psi_delta['Psi'], Si_50nmSiO2_50nmTiO2_psi_delta['Delta'])

In [619]:
TiO2_ = pd.read_csv(os.path.join('..', 'resources', 'TiO2_.csv'), sep='\t', index_col=3).reset_index(drop=True)

In [621]:
Si_50nmSiO2_50nmTiO2_Ns = np.array([N(Air_nk['n'], Air_nk['k']), N(TiO2_['n'], TiO2_['k']), N(SiO2_nk['n'], SiO2_nk['k']), N(Si_nk['n'], Si_nk['k'])])

In [648]:
thicknesses = [0.05, 0.05, 100]
#thetas = [np.deg2rad(60)]
psis3 = []
deltas3 = []
for idx, wavelength in enumerate(Si_50nmSiO2_psi_delta['Wavelength']):
    thetas = [np.deg2rad(60)]
    thetas.extend([theta_j(Si_50nmSiO2_50nmTiO2_Ns[idx_layer][idx], Si_50nmSiO2_50nmTiO2_Ns[idx_layer+1][idx], thetas[-1]) for idx_layer in range(len(Si_50nmSiO2_50nmTiO2_Ns)-1)])
    #print(thetas)

    _M_np = [M_n(beta(thicknesses[i], wavelength, Si_50nmSiO2_50nmTiO2_Ns[i][idx], thetas[i]), rp(Si_50nmSiO2_50nmTiO2_Ns[i][idx], Si_50nmSiO2_50nmTiO2_Ns[i+1][idx], thetas[i], thetas[i+1]), t_p(Si_50nmSiO2_50nmTiO2_Ns[i][idx], Si_50nmSiO2_50nmTiO2_Ns[i+1][idx], thetas[i], thetas[i+1])) for i in range(1, len(Si_50nmSiO2_50nmTiO2_Ns)-2)]

    _M_ns = [M_n(beta(thicknesses[i], wavelength, Si_50nmSiO2_50nmTiO2_Ns[i][idx], thetas[i]), rs(Si_50nmSiO2_50nmTiO2_Ns[i][idx], Si_50nmSiO2_50nmTiO2_Ns[i+1][idx], thetas[i], thetas[i+1]), t_s(Si_50nmSiO2_50nmTiO2_Ns[i][idx], Si_50nmSiO2_50nmTiO2_Ns[i+1][idx], thetas[i], thetas[i+1])) for i in range(1, len(Si_50nmSiO2_50nmTiO2_Ns)-2)]

    _Mp = M(rp(Si_50nmSiO2_50nmTiO2_Ns[0][idx], Si_50nmSiO2_50nmTiO2_Ns[1][idx], thetas[0], thetas[1]), t_p(Si_50nmSiO2_50nmTiO2_Ns[0][idx], Si_50nmSiO2_50nmTiO2_Ns[1][idx], thetas[0], thetas[1]), _M_np)

    _Ms = M(rs(Si_50nmSiO2_50nmTiO2_Ns[0][idx], Si_50nmSiO2_50nmTiO2_Ns[1][idx], thetas[0], thetas[1]), t_s(Si_50nmSiO2_50nmTiO2_Ns[0][idx], Si_50nmSiO2_50nmTiO2_Ns[1][idx], thetas[0], thetas[1]), _M_ns)

    #print(_Ms)

    #print(Sp)

    rp_ = rp(Sp=_Mp)
    rs_ = rs(Ss=_Ms)
    #print(rs_)

    psis3.append(psi(rp_, rs_))
    deltas3.append(delta(rp_, rs_))


In [649]:
psi_delta_plot(Si_50nmSiO2_50nmTiO2_psi_delta['Wavelength'], np.rad2deg(psis3), np.rad2deg(deltas3))